In [1]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ID Сайта], [Артикул поставщика], [Товарная группа], [Товарное направление], [Название товара], [Код склада], [Остаток], [Код товара]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ЦБ', N'СИ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [4]:
stock = stock[stock['ID Сайта'] != '']

In [7]:
ost = stock.groupby(by = 'ID Сайта', as_index = False)['Остаток'].sum()

In [11]:
count = stock.groupby(by=['ID Сайта'], as_index=False)['Код объекта'].nunique()

In [14]:
ost = ost.merge(count, how='left', on='ID Сайта')

In [16]:
ost.rename(columns={'Код объекта': 'На каком кол-ве действующих объектов есть остаток'}, inplace=True)

In [17]:
stock.drop(columns=['Остаток', 'Код объекта', 'Код склада'], inplace=True)

In [18]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\ecomsales')
extension = 'xlsx'
all_sales = [i for i in glob.glob('*.{}'.format(extension))]

sale = pd.DataFrame()
for name in tqdm(all_sales):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [02:57<00:00, 11.07s/it]


In [19]:
sale = sale.reset_index(drop=True)
sale.drop_duplicates(inplace=True)

sale = sale[(sale['state_order'] != 'returned') & (sale['state_order'] != 'canceled')]
sale = sale[~((sale['delivery_type'] == 'PICKUP_NOW') & (sale['payment_type'] == 'CASHIER') & (sale['type_ibox'] == 'CASHIER'))|
                ((sale['delivery_type'] == 'PICKUP') & (sale['payment_type'] == 'CASHIER') & (sale['type_ibox'] == 'CASHIER'))]

In [20]:
sale = sale[~sale['client_type'].str.contains('OZON')]

In [21]:
sale['result'] = sale['price_wo_manzana']
sale['result'].fillna(sale['price'], inplace=True)

In [22]:
def str_float(row):
    row = str(row)
    row = row.replace(',','.')
    return row

sale['price'] = sale['price'].apply(str_float)
sale['price'] = sale['price'].astype(float)
sale = sale[sale['price'] > 1].reset_index(drop=True)

In [23]:
sebes = sale.groupby(by = 'tov', as_index=False)['cost_price'].sum()

In [24]:
online = sale.groupby(by = 'tov', as_index=False)['result'].agg({'sum', 'count'})

In [25]:
online = online.merge(sebes, how='left', on='tov')

In [26]:
online.rename(columns={'tov': 'Код товара', 'sum': 'Сумма продаж', 'count': 'Количество продаж', 'cost_price': 'Себестоимость'}, inplace=True)

In [27]:
online

,Код товара,Количество продаж,Сумма продаж,Себестоимость
0,ТОВ0115302,1,33764.000000,15031.88
1,ТОВ0115418,1,4292.100000,1315.80
2,ТОВ0120217,1,8424.000000,3680.03
3,ТОВ0120218,1,8977.500000,4004.34
4,ТОВ0120219,1,16906.200000,4278.07
...,...,...,...,...
82689,ТОВ1329932,1,2450.000000,954.99
82690,ТОВ1329934,1,5480.000000,2355.00
82691,ТОВ1329935,1,5480.000000,2355.00
82692,ТОВ1330947,1,2666.689557,1820.48


In [28]:
sale = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Продажи по ТОВ СИ, ЦБ янв.23-июн.24.xlsx')

In [29]:
sale.rename(columns={'Номер': 'Код товара'}, inplace=True)

In [30]:
sale = sale.merge(online, how='left', on='Код товара')

In [31]:
sale

,Код товара,Количество-продажи,Сумма продаж с баллами,Себестоимость продаж,Количество продаж,Сумма продаж,Себестоимость
0,ТОВ0115302,5,198154.16,65596.85,1.0,33764.0,15031.88
1,ТОВ0115418,1,4292.10,1315.80,1.0,4292.1,1315.80
2,ТОВ0115554,1,66654.28,15108.45,NaN,NaN,NaN
3,ТОВ0119984,1,11000.00,2060.18,NaN,NaN,NaN
4,ТОВ0120217,1,8424.00,3680.03,1.0,8424.0,3680.03
...,...,...,...,...,...,...,...
104956,ТОВ1335466,2,10449.30,7512.00,NaN,NaN,NaN
104957,ТОВ1335467,4,23400.00,15412.00,NaN,NaN,NaN
104958,ТОВ1335468,1,6060.00,3986.00,NaN,NaN,NaN
104959,ТОВ1335469,2,12920.00,8498.00,NaN,NaN,NaN


In [32]:
sale.fillna(0, inplace=True)

In [33]:
sale['Сумма'] = sale['Сумма продаж'] + sale['Сумма продаж с баллами']
sale['Себес'] = sale['Себестоимость продаж'] + sale['Себестоимость']
sale['Кол-во'] = sale['Количество продаж'] + sale['Количество-продажи']

In [34]:
sale.drop(columns= ['Количество-продажи', 'Сумма продаж с баллами', 'Себестоимость продаж', 'Сумма продаж', 'Количество продаж', 'Себестоимость'],
        inplace = True)

In [35]:
stock = stock.merge(sale, how='left', on='Код товара')

In [36]:
stock = stock.merge(ost, how='left', on='ID Сайта')

In [37]:
stock.drop_duplicates(subset=['Код товара'], inplace=True)

In [38]:
stock.drop(columns=['Код товара'], inplace=True)

In [39]:
stock.fillna(0, inplace=True)

In [40]:
sales_total = stock.groupby(by=['ID Сайта', 'Артикул поставщика', 'Товарная группа', 'Товарное направление', 'Название товара', 'Остаток', 'На каком кол-ве действующих объектов есть остаток'], 
                            as_index=False)[['Себес', 'Кол-во',  'Сумма']].sum()

In [41]:
sales_total.drop_duplicates(subset=['ID Сайта'], inplace=True)

In [64]:
cb = sales_total[sales_total['Товарное направление'] == 'ЦБ']
si = sales_total[sales_total['Товарное направление'] == 'СИ']

In [65]:
cb.sort_values(by=['Сумма'], ascending=False, inplace=True)
si.sort_values(by=['Сумма'], ascending=False, inplace=True)

In [67]:
si.reset_index(inplace=True)
cb.reset_index(inplace=True)

In [69]:
cb = cb[:100]
si = si[:100]

In [71]:
si.drop(columns=['index'], inplace=True)
cb.drop(columns=['index'], inplace=True)

In [72]:
si['Приоритет'] = si.index + 1
cb['Приоритет'] = si.index + 1

In [87]:
result = pd.concat([cb, si], ignore_index=True)

In [88]:
result['ID Сайта'] = result['ID Сайта'].astype(str)
result['Ссылка на сайт'] = 'https://www.585zolotoy.ru/catalog/products/' + result['ID Сайта'] + '/'

In [89]:
result.rename(columns={
    'ID Сайта': 'ID сайта',
    'Артикул поставщика': 'Артикул товара',
    'Товарное направление': 'ТН',
    'Остаток': 'Текущий остаток по складам действующих объектов',
    'Кол-во': 'Кол-во продаж',
    'Сумма': 'Сумма продаж',
    'Себес': 'Себестоимость продаж',
    'Название товара': 'Описание товара'
}, inplace=True)

In [92]:
result.columns

Index(['ID сайта', 'Артикул товара', 'Товарная группа', 'ТН',
       'Описание товара', 'Текущий остаток по складам действующих объектов',
       'На каком кол-ве действующих объектов есть остаток',
       'Себестоимость продаж', 'Кол-во продаж', 'Сумма продаж', 'Приоритет',
       'Ссылка на сайт', 'Наценка, %', 'Цена 1 шт, руб'],
      dtype='object')

In [91]:
result['Наценка, %'] = result['Сумма продаж'] / result['Себестоимость продаж'] - 1 
result['Цена 1 шт, руб'] = result['Сумма продаж'] / result['Кол-во продаж']

In [94]:
result = result[['Ссылка на сайт', 'Артикул товара', 'Товарная группа', 'ТН', 'Приоритет', 
                'Описание товара', 'ID сайта', 'Текущий остаток по складам действующих объектов', 
                'На каком кол-ве действующих объектов есть остаток', 'Кол-во продаж',
                'Сумма продаж', 'Себестоимость продаж', 'Наценка, %', 'Цена 1 шт, руб']]

In [95]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\photo\image_parser\dataset\result.xlsx', index=False)